In [9]:
import numpy as np
from dolfinx import mesh, fem, io
from mpi4py import MPI
from petsc4py import PETSc
import ufl

# Parameters
length = 10e-6
height = 5e-6
nx = 100
ny = 50

# 2D mesh
domain = mesh.create_rectangle(MPI.COMM_WORLD,
                               [np.array([0, 0]), np.array([length, height])],
                               [nx, ny],
                               cell_type=mesh.CellType.triangle)

# Function space
V = fem.functionspace(domain, ("CG", 1))

# Relative permittivity (scalar)
eps_r = fem.Constant(domain, PETSc.ScalarType(28.0))  # Approx LiNbO3

# Boundary markers
def left_boundary(x):
    return np.isclose(x[0], 0.0)

def right_boundary(x):
    return np.isclose(x[0], length)

facets_left = mesh.locate_entities_boundary(domain, dim=1, marker=left_boundary)
facets_right = mesh.locate_entities_boundary(domain, dim=1, marker=right_boundary)

# Dirichlet BCs
dofs_left = fem.locate_dofs_topological(V, entity_dim=1, entities=facets_left)
dofs_right = fem.locate_dofs_topological(V, entity_dim=1, entities=facets_right)

bc_left = fem.dirichletbc(PETSc.ScalarType(100.0), dofs_left, V)
bc_right = fem.dirichletbc(PETSc.ScalarType(0.0), dofs_right, V)

# Variational problem
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

a = eps_r * ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx(domain)
# Linear form (homogeneous RHS)
zero = fem.Constant(domain, PETSc.ScalarType(0.0))
L = zero * v * ufl.dx(domain)

# LinearProblem setup
from dolfinx.fem.petsc import LinearProblem

problem = LinearProblem(
    a,
    L,
    bcs=[bc_left, bc_right],
    petsc_options={"ksp_type": "cg"}
)

phi_sol = problem.solve()

In [10]:
# Compute E = -grad(phi)
E = fem.Expression(-ufl.grad(phi_sol), V.element.interpolation_points())

ld: warning: duplicate -rpath '/opt/anaconda3/envs/fenicsx-env/lib' ignored
ld: warning: duplicate -rpath '/opt/anaconda3/envs/fenicsx-env/lib' ignored


In [12]:
# Export to XDMF
with io.XDMFFile(MPI.COMM_WORLD, "potential.xdmf", "w") as xdmf:
    xdmf.write_mesh(domain)
    phi_sol.name = "Potential"
    xdmf.write_function(phi_sol)

# Save E-field components
# Note: Since E is a vector expression, project each component
W = fem.functionspace(domain, ("CG", 1))
grad_phi = ufl.grad(phi_sol)

# Project Ex
Ex = fem.Function(W)
ex_form = -grad_phi[0]
a_proj = ufl.inner(ufl.TestFunction(W), ufl.TrialFunction(W)) * ufl.dx
L_proj_ex = ufl.inner(ufl.TestFunction(W), ex_form) * ufl.dx
problem_ex = LinearProblem(a_proj, L_proj_ex, petsc_options={"ksp_type": "cg"})
Ex.x.array[:] = problem_ex.solve().x.array

# Project Ey
Ey = fem.Function(W)
ey_form = -grad_phi[1]
L_proj_ey = ufl.inner(ufl.TestFunction(W), ey_form) * ufl.dx
problem_ey = LinearProblem(a_proj, L_proj_ey, petsc_options={"ksp_type": "cg"})
Ey.x.array[:] = problem_ey.solve().x.array

with io.XDMFFile(MPI.COMM_WORLD, "efield.xdmf", "w") as xdmf:
    xdmf.write_mesh(domain)
    Ex.name = "Ex"
    Ey.name = "Ey"
    xdmf.write_function(Ex)
    xdmf.write_function(Ey)

ld: warning: duplicate -rpath '/opt/anaconda3/envs/fenicsx-env/lib' ignored
ld: warning: duplicate -rpath '/opt/anaconda3/envs/fenicsx-env/lib' ignored
ld: warning: duplicate -rpath '/opt/anaconda3/envs/fenicsx-env/lib' ignored
ld: warning: duplicate -rpath '/opt/anaconda3/envs/fenicsx-env/lib' ignored
ld: warning: duplicate -rpath '/opt/anaconda3/envs/fenicsx-env/lib' ignored
ld: warning: duplicate -rpath '/opt/anaconda3/envs/fenicsx-env/lib' ignored
